### Notes 

T5 Paper: https://arxiv.org/pdf/1910.10683.pdf

T5 Tokenizer: https://github.com/huggingface/transformers/blob/master/src/transformers/tokenization_t5.py

Important Tasks: https://docs.google.com/document/d/1weIZM6QTlnitpPQmpg-WeV2RW70TnYmDuogBQPr5mB0/edit

In [1]:
#installation step
!pip install transformers
!pip install t5
!pip install sentencepiece
!pip install bpemb
#creating the folders 
!mkdir data/
!mkdir data/AD_NMT-master
!mkdir data/train/
!mkdir data/test/
!mkdir data/val/
!mkdir data/vocab/
!mkdir data/model/
#fetching the pkl files
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW' -O data/AD_NMT-master/english-Arabic-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V8_tp8ZlWUYaX7QQL46t0uSRNrVehSf1' -O data/AD_NMT-master/english-Arabic-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V7X0qtuDIyjTHY0wh-ZNoVwsiF4lId2e' -O data/AD_NMT-master/english-Arabic-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UzL4cOWTMCee83KBUh2QO_H62AFVpDQV' -O data/AD_NMT-master/LAV-MSA-2-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UpfCbkxhztof7dvNjeAs1bHjD4SER6h3' -O data/AD_NMT-master/LAV-MSA-2-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UlAZGtYsSfXzK7hrC_PbxQFqTSXD0DMw' -O data/AD_NMT-master/LAV-MSA-2-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UjDX7cCG2S23SPfSHxSPdVayMTxB5Y16' -O data/AD_NMT-master/Magribi_MSA-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UaVWIqRXo0rxuxDF4KArA4bEK1TaLX3l' -O data/AD_NMT-master/Magribi_MSA-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UYvlhdYAdfa4riP_4hn3-IEVd1ZUXVTQ' -O data/AD_NMT-master/Magribi_MSA-train.pkl

     |████████████████████████████████| 573kB 9.2MB/s 
     |████████████████████████████████| 3.7MB 64.0MB/s 
     |████████████████████████████████| 1.0MB 54.9MB/s 
     |████████████████████████████████| 890kB 47.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=5873e9517a1216a5b50b788d098504e20133494faf8edfefafc5fa675f3f3811
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses
     |████████████████████████████████| 143kB 8.2MB/s 
     |████████████████████████████████| 296kB 10.2MB/s 
     |████████████████████████████████| 8.3MB 12.6MB/s 
     |████████████████████████████████| 61kB 10.7MB/s 
     |████████████████████████████████| 3.3MB 64.2MB/s 
     |████████████████████████████████| 421.8MB 31kB/s 
     |████████████████████████████████| 3.9MB 49.5MB/s 
     |████████████████████████████████| 450kB 52.2MB/s 
  Created wheel for gast: filenam

--2020-05-03 10:03:09--  https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW
Resolving docs.google.com (docs.google.com)... 74.125.143.139, 74.125.143.100, 74.125.143.138, ...
Connecting to docs.google.com (docs.google.com)|74.125.143.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-2s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/snbujfhv25q5f57p184ket50sg8trdc8/1588500150000/16970776037313924126/*/1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW?e=download [following]
--2020-05-03 10:03:09--  https://doc-10-2s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/snbujfhv25q5f57p184ket50sg8trdc8/1588500150000/16970776037313924126/*/1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW?e=download
Resolving doc-10-2s-docs.googleusercontent.com (doc-10-2s-docs.googleusercontent.com)... 108.177.126.132, 2a00:1450:4013:c01::84
Connecting to doc-10-2s-docs.googleusercontent.com (doc-10-2s-

In [0]:
#James Chartouni
#Joey Park
#Raef Khan

import torch
import pandas as pd
import numpy as np
import pickle
import os, io, glob
import functools

import sentencepiece as spm
from bpemb import BPEmb

import transformers
import t5
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split

##Initial Loading from Pickle

In [0]:
VOCAB_SIZE = 5000

In [4]:
ls data/AD_NMT-master

english-Arabic-both.pkl   LAV-MSA-2-both.pkl   Magribi_MSA-both.pkl
english-Arabic-test.pkl   LAV-MSA-2-test.pkl   Magribi_MSA-test.pkl
english-Arabic-train.pkl  LAV-MSA-2-train.pkl  Magribi_MSA-train.pkl


In [0]:
file_path = 'data/AD_NMT-master/'

with open(file_path + "english-Arabic-train.pkl", 'rb') as handle:
    data_English_MSA_trainval = pickle.load(handle)

with open(file_path + "english-Arabic-test.pkl", 'rb') as handle:
    data_English_MSA_test = pickle.load(handle)

with open(file_path + "english-Arabic-both.pkl", 'rb') as handle:
    data_English_MSA_both = pickle.load(handle) 




with open(file_path + "LAV-MSA-2-train.pkl", 'rb') as handle:
    data_LAV_MSA_trainval = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-test.pkl", 'rb') as handle:
    data_LAV_MSA_test = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-both.pkl", 'rb') as handle:
    data_LAV_MSA_both = pickle.load(handle) 




with open(file_path + "Magribi_MSA-train.pkl", 'rb') as handle:
    data_Magribi_MSA_trainval = pickle.load(handle) 
    
with open(file_path + "Magribi_MSA-test.pkl", 'rb') as handle:
    data_Magribi_MSA_test = pickle.load(handle) 

with open(file_path + "Magribi_MSA-both.pkl", 'rb') as handle:
    data_Magribi_MSA_both = pickle.load(handle) 
    

In [6]:
#few dataset examples
print(data_English_MSA_both[0:5])
print(data_LAV_MSA_both[0:5])

#print length of train + val dataset, print length of both (train + val) + test
print(len(data_English_MSA_trainval))
print(len(data_English_MSA_both))

[['Tom was also there', 'كان توم هنا ايضا'], ['That old woman lives by herself', 'تلك المراة العجوز تسكن بمفردها'], ['He went abroad for the purpose of studying English', 'سافر خارج البلد ليتعلم الانجليزية'], ['There is a fork missing', 'هناك شوكة ناقصة'], ["I don't know this game", 'لا اعرف هذه اللعبة']]
[['لا انا بعرف وحدة راحت ع فرنسا و معا شنتا حطت فيها الفرش', 'لا اعرف واحدة ذهبت الى فرنسا و لها غرفة و ضعت فيها الافرشة'], ['روح بوشك و فتول عاليسار', 'اذهب تقدم و استدر يسارا'], ['لا لا لازم انه يكون عندك موضوع ما في اشي', ' لا لا يجب ان يكون لديك موضوع هذا ضروري'], ['اوعي تبعدي من هون بلاش تضيعي ', 'لا تبتعد عن هنا حتى لا تفقد الطريق '], ['قصدي صراحة يما انا كمان كرهته من يوم ما عملتيه زي ما بتعمله خالتي كرهته و صرت ما باطيقه بالمرة', 'اقصد صراحة يا امي انا ايضا كرهته من يوم حضرته مثلما تحضره خالتي كرهته و اصبحت لا اطيقه ابدا']]
9000
10001


## Prepare Datasets

example: https://iwslt2010.fbk.eu/node/32/

We need to take our training and test sets from the pkl files and create new .txt files that are formatted so that the standard torchtext Dataset class can read them

Data format:
each line consists of three fields divided by the character '\'
sentences consisting of words divided by single spaces
format: <SENTENCE_ID>\<PARAPHRASE_ID>\<TEXT>
Field_1: sentence ID
Field_2: paraphrase ID
Field_3: MT develop sentence / reference translation
Text input example:
DEV_001\01\This is the first develop sentence.
DEV_002\01\This is the second develop sentence.
Reference translation example:
DEV_001\01\1st reference translation for 1st input
DEV_001\02\2nd reference translation for 1st input
...
DEV_002\01\1st reference translation for 2nd input
DEV_002\02\2nd reference translation for 2nd input
...
Languages:
Arabic-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
IWSLT07 testset: 489 sentences, 6 reference translations
IWSLT08 testset: 507 sentences, 16 reference translations
French-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
Turkish-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations


In [7]:
ls data/AD_NMT-master/

english-Arabic-both.pkl   LAV-MSA-2-both.pkl   Magribi_MSA-both.pkl
english-Arabic-test.pkl   LAV-MSA-2-test.pkl   Magribi_MSA-test.pkl
english-Arabic-train.pkl  LAV-MSA-2-train.pkl  Magribi_MSA-train.pkl


In [0]:
#splits the train dataset into train and validation sets, define test set as datafile
en_msa_train, en_msa_val = train_test_split(data_English_MSA_trainval, test_size=.2)
en_msa_test = data_English_MSA_test

lav_msa_train, lav_msa_val = train_test_split(data_LAV_MSA_trainval, test_size=.2)
lav_msa_test = data_LAV_MSA_test

mag_msa_train, mag_msa_val = train_test_split(data_Magribi_MSA_trainval, test_size=.2)
mag_msa_test = data_Magribi_MSA_test

In [9]:
print(len(en_msa_train))
print(len(en_msa_val))

print(len(lav_msa_train))
print(len(lav_msa_val))

print(len(mag_msa_train))
print(len(mag_msa_val))

7200
1800
11044
2761
14188
3548


In [0]:
file_path = 'data/'

def list_to_csv(ds, src='en', trg='msa', datatype=''):
    src_formatted = datatype + '_' + src + '_' + trg + '.' + 'csv'
    
    with open(file_path + datatype + "/" + src_formatted, 'wt') as csv:
        for i, arr in enumerate(ds):
            csv.write(arr[0] + ',' + arr[1] + '\n')

In [0]:
list_to_csv(en_msa_train, 'en', 'msa', 'train')
list_to_csv(en_msa_val, 'en', 'msa', 'val')
list_to_csv(en_msa_test, 'en', 'msa', 'test')

list_to_csv(lav_msa_train, 'lav', 'msa', 'train')
list_to_csv(lav_msa_val, 'lav', 'msa', 'val')
list_to_csv(lav_msa_test, 'lav', 'msa', 'test')

list_to_csv(mag_msa_train, 'mag', 'msa', 'train')
list_to_csv(mag_msa_val, 'mag', 'msa', 'val')
list_to_csv(mag_msa_test, 'mag', 'msa', 'test')

## Build Vocabulary 

Sentence Piece Google Colab
https://github.com/google/sentencepiece/blob/master/python/sentencepiece_python_module_example.ipynb

In [0]:
# """
# Testing:
# Create a text file with all the vocab available from all sources for each language for SentencePiece to create a library 

# TODO: implement bpemb model for MSA vocab/train SPM on larger datasets for dialects
# """

# en_vocab = open("data/vocab/en_vocab.txt", "wt")
# msa_vocab = open("data/vocab/msa_vocab.txt", "wt")
# lav_vocab = open("data/vocab/lav_vocab.txt", "wt")
# mag_vocab = open("data/vocab/mag_vocab.txt", "wt")

# MSA_text = ""
# EN_text = ""

# def create_vocab(file='', src='en_vocab', tgt='msa_vocab'):
#   for line in file:
#         src_sent = line[0]
#         src_words = src_sent.split(" ")
#         for count, word in enumerate(src_words):
#             src.write(word)
#         src.write("\n")
        
#         tgt_sent = line[1]
#         tgt_words = tgt_sent.split(" ")
#         for count, word in enumerate(tgt_words):
#             tgt.write(word)
#         tgt.write("\n")

# create_vocab(data_English_MSA_both, en_vocab, msa_vocab)
# create_vocab(data_LAV_MSA_both, lav_vocab, msa_vocab)
# create_vocab(data_Magribi_MSA_both, mag_vocab, msa_vocab)

# en_vocab.close()
# msa_vocab.close()
# lav_vocab.close()
# mag_vocab.close()

In [0]:
#spm.SentencePieceTrainer.train('--input=data/vocab/en_vocab.txt,data/vocab/msa_vocab.txt,data/vocab/lav_vocab.txt,data/vocab/mag_vocab.txt --model_prefix=data/model/spm --vocab_size=' + str(VOCAB_SIZE))

In [0]:
# ls data/model

In [0]:
# sp = spm.SentencePieceProcessor()
# sp.load('data/model/spm.model')

In [0]:
# print(sp.encode_as_pieces('This is a test'))
# print(sp.encode_as_ids('This is a test'))

##Loading MultiBPemb: Multilingual BPEmb Sentencepiece Model

In [17]:
multibpemb = BPEmb(lang="multi", vs=1000000, dim=300)

downloading https://nlp.h-its.org/bpemb/multi/multi.wiki.bpe.vs1000000.model


  0%|          | 3696640/1123372891 [00:00<00:30, 36965025.15B/s]

downloading https://nlp.h-its.org/bpemb/multi/multi.wiki.bpe.vs1000000.d300.w2v.bin.tar.gz


100%|██████████| 1123372891/1123372891 [00:14<00:00, 76205918.07B/s]
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


##Tensor Processing + Add to TaskRegistry

### English to Arabic Task

In [0]:
en_msa_split_csv_path = {
    "train": "data/train/train_en_msa.csv",
    "validation": "data/val/val_en_msa.csv"
}
en_msa_example_count = {
    "train": 7200,
    "validation": 1800
}

In [19]:
def en_msa_translation_dataset_fn(split, shuffle_files=False):
  ds = tf.data.TextLineDataset(en_msa_split_csv_path[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["",""],
                        field_delim=",", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE
  )
  ds = ds.map(lambda *example: dict(zip(["source", "target"], example)) )
  return ds

for example in tfds.as_numpy(en_msa_translation_dataset_fn("train").take(5)):
    print(example)

{'source': b'How can you know that?', 'target': b'\xd9\x83\xd9\x8a\xd9\x81 \xd8\xa8\xd8\xa7\xd9\x85\xd9\x83\xd8\xa7\xd9\x86\xd9\x83 \xd9\x85\xd8\xb9\xd8\xb1\xd9\x81\xd8\xa9 \xd8\xb0\xd9\x84\xd9\x83\xd8\x9f'}
{'source': b'Tom picked up the bottle of shampoo and squirted some in his hand', 'target': b'\xd8\xaa\xd9\x88\xd9\x85 \xd8\xa7\xd9\x84\xd8\xaa\xd9\x82\xd8\xb7 \xd8\xa8\xd8\xb7\xd9\x84 \xd8\xa7\xd9\x84\xd8\xb4\xd8\xa7\xd9\x85\xd8\xa8\xd9\x88 \xd9\x88 \xd8\xa8\xd8\xae \xd8\xa7\xd9\x84\xd9\x82\xd9\x84\xd9\x8a\xd9\x84 \xd9\x81\xd9\x8a \xd9\x8a\xd8\xaf\xd9\x87'}
{'source': b'Tom has a dentist appointment at 230', 'target': b'\xd8\xaa\xd9\x88\xd9\x85 \xd9\x84\xd8\xaf\xd9\x8a\xd9\x87 \xd9\x85\xd9\x88\xd8\xb9\xd8\xaf \xd8\xa7\xd8\xb3\xd9\x86\xd8\xa7\xd9\x86 \xd8\xb9\xd9\x86\xd8\xaf \xd8\xa7\xd9\x84\xd8\xab\xd8\xa7\xd9\x86\xd9\x8a\xd8\xa9 \xd9\x88\xd8\xa7\xd9\x84\xd9\x86\xd8\xb5\xd9\x81'}
{'source': b'A rabbit has long ears', 'target': b'\xd9\x84\xd9\x84\xd8\xa7\xd8\xb1\xd9\x86\xd8\xa8 \xd8

In [0]:
#turn the ds of dictionaries and change the keys to inputs and targets that the model
def en_msa_translation_preprocessor(ds):
  def to_inputs_and_targets(ex):
    return{
        "inputs": tf.strings.join(["Translate English to MSA: ",ex["source"]]),
        "targets": ex["target"]
    }
  return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [0]:
t5.data.TaskRegistry.remove("en_msa_translation")
t5.data.TaskRegistry.add(
    #name of the Task
    "en_msa_translation",
    #Supply a function which returns a tf.data.Dataset
    dataset_fn=en_msa_translation_dataset_fn,
    splits=["train", "validation"],
    # Supply a function which preprocesses text from the tf.data.Dataset.
    text_preprocessor=[en_msa_translation_preprocessor],
    # Use the same vocabulary that we used for pre-training.
    sentencepiece_model_path= t5.data.DEFAULT_SPM_PATH, #str(multibpemb.model_file),
    # Lowercase targets before computing metrics.
    postprocess_fn = t5.data.postprocessors.lower_text, 
    # We'll use accuracy as our evaluation metric.
    metric_fns=[t5.evaluation.metrics.accuracy],
    # Not required, but helps for mixing and auto-caching.
    #num_input_examples=num_nq_examples
)

###Levantine to MSA Task

In [0]:
lav_msa_split_csv_path = {
    "train": "data/train/train_lav_msa.csv",
    "validation": "data/val/val_lav_msa.csv"
}
lav_msa_example_count = {
    "train": 11044,
    "validation": 2761
}

In [23]:
def lav_msa_translation_dataset_fn(split, shuffle_files=False):
  ds = tf.data.TextLineDataset(lav_msa_split_csv_path[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["",""],
                        field_delim=",", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE
  )
  ds = ds.map(lambda *example: dict(zip(["source", "target"], example)) )
  return ds

for example in tfds.as_numpy(lav_msa_translation_dataset_fn("train").take(5)):
    print(example)

{'source': b'\xd8\xa7\xd9\x84\xd9\x84\xd9\x87 \xd9\x8a\xd9\x86\xd8\xb5\xd8\xb1 \xd8\xa7\xd9\x84\xd8\xa7\xd8\xb3\xd9\x84\xd8\xa7\xd9\x85 \xd9\x88\xd8\xa7\xd9\x84\xd9\x85\xd8\xb3\xd9\x84\xd9\x85\xd9\x8a\xd9\x86  \xd8\xa7\xd9\x84\xd9\x84\xd9\x87 \xd9\x8a\xd8\xac\xd8\xb2\xd9\x8a\xd9\x83\xd9\x85 \xd9\x83\xd9\x84 \xd8\xae\xd9\x8a\xd8\xb1 ', 'target': b'\xd8\xb1\xd8\xa8\xd9\x86\xd8\xa7 \xd9\x8a\xd9\x86\xd8\xb5\xd8\xb1 \xd8\xa7\xd9\x84\xd8\xa7\xd8\xb3\xd9\x84\xd8\xa7\xd9\x85 \xd9\x88\xd8\xa7\xd9\x84\xd9\x85\xd8\xb3\xd9\x84\xd9\x85\xd9\x8a\xd9\x86 \xd8\xac\xd8\xb2\xd8\xa7\xd9\x83\xd9\x85 \xd8\xa7\xd9\x84\xd9\x84\xd9\x87 \xd9\x83\xd9\x84 \xd8\xae\xd9\x8a\xd8\xb1'}
{'source': b'\xd8\xa7\xd9\x84\xd9\x85\xd9\x87\xd9\x85 \xd8\xa7\xd9\x84\xd8\xac\xd9\x88 \xd9\x83\xd9\x84\xd9\x88 \xd8\xa7\xd9\x84\xd9\x8a\xd9\x88\xd9\x85 \xd8\xa7\xd8\xad\xd9\x85\xd8\xb1 \xd9\x81\xd9\x8a \xd8\xa7\xd8\xad\xd9\x85\xd8\xb1 \xd9\x88\xd8\xac\xd9\x86\xd8\xa7\xd9\x86', 'target': b'\xd8\xa7\xd9\x84\xd8\xb4\xd9\x89\xd8\xa1 \xd8\

In [0]:
#turn the ds of dictionaries and change the keys to inputs and targets that the model
def lav_msa_translation_preprocessor(ds):
  def to_inputs_and_targets(ex):
    return{
        "inputs": tf.strings.join(["Translate Levantine to MSA: ",ex["source"]]),
        "targets": ex["target"]
    }
  return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [0]:
t5.data.TaskRegistry.remove("lav_msa_translation")
t5.data.TaskRegistry.add(
    #name of the Task
    "lav_msa_translation",
    #Supply a function which returns a tf.data.Dataset
    dataset_fn=lav_msa_translation_dataset_fn,
    splits=["train", "validation"],
    # Supply a function which preprocesses text from the tf.data.Dataset.
    text_preprocessor=[lav_msa_translation_preprocessor],
    # Use the same vocabulary that we used for pre-training.
    sentencepiece_model_path=t5.data.DEFAULT_SPM_PATH, #str(multibpemb.model_file),
    # Lowercase targets before computing metrics.
    postprocess_fn = t5.data.postprocessors.lower_text, 
    # We'll use accuracy as our evaluation metric.
    metric_fns=[t5.evaluation.metrics.accuracy],
    # Not required, but helps for mixing and auto-caching.
    #num_input_examples=num_nq_examples
)

###Maghrib to MSA Task

In [0]:
mag_msa_split_csv_path = {
    "train": "data/train/train_mag_msa.csv",
    "validation": "data/val/val_mag_msa.csv"
}
mag_msa_example_count = {
    "train": 14188,
    "validation": 3548
}

In [27]:
def mag_msa_translation_dataset_fn(split, shuffle_files=False):
  ds = tf.data.TextLineDataset(mag_msa_split_csv_path[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["",""],
                        field_delim=",", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE
  )
  ds = ds.map(lambda *example: dict(zip(["source", "target"], example)) )
  return ds

for example in tfds.as_numpy(mag_msa_translation_dataset_fn("train").take(5)):
    print(example)

{'source': b'\xd8\xaa\xd9\x81\xd8\xb1\xd8\xac\xd9\x8a \xd9\x81\xd9\x8a \xd9\x87\xd8\xa7\xd8\xaf \xd8\xa7\xd9\x84\xd8\xad\xd9\x84\xd9\x82\xd8\xa9 \xd8\xb1\xd8\xa7\xd9\x87\xd8\xa7 \xd8\xac\xd8\xaf\xd9\x8a\xd8\xaf\xd8\xa9', 'target': b'\xd8\xb4\xd8\xa7\xd9\x87\xd8\xaf\xd9\x8a \xd9\x87\xd8\xb0\xd9\x87 \xd8\xa7\xd9\x84\xd8\xad\xd9\x84\xd9\x82\xd8\xa7\xd8\xaa \xd8\xa7\xd9\x86\xd9\x87\xd8\xa7 \xd8\xac\xd9\x8a\xd8\xaf\xd8\xa9'}
{'source': b'\xd8\xb5\xd8\xa8\xd8\xa7\xd8\xad \xd8\xa7\xd9\x84\xd8\xae\xd9\x8a\xd8\xb1 \xd8\xb4\xd9\x83\xd9\x88\xd9\x86 \xd9\x85\xd8\xb9\xd8\xa7\xd9\x8a\xd8\xa7', 'target': b'\xd8\xb5\xd8\xa8\xd8\xa7\xd8\xad \xd8\xa7\xd9\x84\xd8\xae\xd9\x8a\xd8\xb1 \xd9\x85\xd9\x86 \xd9\x85\xd8\xb9\xd9\x8a\xd8\x9f'}
{'source': b' \xd8\xaf\xd8\xb1\xd9\x83 \xd9\x86\xd8\xb4\xd9\x88\xd9\x81 \xd9\x86\xd8\xa8\xd9\x8a\xd9\x84 \xef\xbb\xa7\xd9\x82\xd9\x88\xd9\x84 \xd9\x84\xd9\x87 \xd9\x83\xd9\x8a \xd9\x8a\xd8\xb1\xd9\x88\xd8\xad \xd9\x8a\xd8\xaf\xd9\x8a \xd8\xb1\xd8\xa4\xd9\x89 \xd9\x86\xd9\x87

In [0]:
#turn the ds of dictionaries and change the keys to inputs and targets that the model
def mag_msa_translation_preprocessor(ds):
  def to_inputs_and_targets(ex):
    return{
        "inputs": tf.strings.join(["Translate Maghrib to Arabic: ",ex["source"]]),
        "targets": ex["target"]
    }
  return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [0]:
t5.data.TaskRegistry.remove("mag_msa_translation")
t5.data.TaskRegistry.add(
    #name of the Task
    "mag_msa_translation",
    #Supply a function which returns a tf.data.Dataset
    dataset_fn=mag_msa_translation_dataset_fn,
    splits=["train", "validation"],
    # Supply a function which preprocesses text from the tf.data.Dataset.
    text_preprocessor=[mag_msa_translation_preprocessor],
    # Use the same vocabulary that we used for pre-training.
    sentencepiece_model_path=t5.data.DEFAULT_SPM_PATH, #str(multibpemb.model_file),
    # Lowercase targets before computing metrics.
    postprocess_fn = t5.data.postprocessors.lower_text, 
    # We'll use accuracy as our evaluation metric.
    metric_fns=[t5.evaluation.metrics.accuracy],
    # Not required, but helps for mixing and auto-caching.
    #num_input_examples=num_nq_examples
)

##Dataset Mixture

In [0]:
t5.data.MixtureRegistry.remove("ar_translation")
t5.data.MixtureRegistry.add(
    "ar_translation",
    ["en_msa_translation", "lav_msa_translation", "mag_msa_translation"],
     default_rate=1.0
)

##Fine Tune

In [32]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

#Using the Huggingface T5 Model
model = t5.models.HfPyTorchModel("t5-base", "/tmp/hft5/", device)

In [38]:
STEPS = 1000 #@param {type: "integer"}
model.train(
    mixture_or_task_name="ar_translation",
    steps=STEPS,
    save_steps=100,
    sequence_length={"inputs": 64, "targets": 4},
    split="train",
    batch_size=32,
    optimizer=functools.partial(transformers.AdamW, lr=1e-4),
)

INFO:absl:Saving checkpoint for step 0
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)
INFO:absl:Saving checkpoint for step 100
INFO:absl:Saving checkpoint for step 200
INFO:absl:Saving checkpoint for step 300
INFO:absl:Saving checkpoint for step 400
INFO:absl:Saving checkpoint for step 500
INFO:absl:Saving checkpoint for step 600
INFO:absl:Saving checkpoint for step 700
INFO:absl:Saving checkpoint for step 800
INFO:absl:Saving checkpoint for step 900
INFO:absl:Saving final checkpoint for step 1000


##Evaluation

In [0]:
# Evaluate after fine-tuning
model.eval(
    "ar_translation",
    checkpoint_steps="all",
    sequence_length={"inputs": 64, "targets": 4},
    batch_size=128,
)

INFO:absl:Loading from /tmp/hft5/model-0.checkpoint
INFO:absl:eval/en_msa_translation/accuracy at step 0: 0.000
INFO:absl:eval/lav_msa_translation/accuracy at step 0: 0.000
INFO:absl:eval/mag_msa_translation/accuracy at step 0: 0.000
INFO:absl:Loading from /tmp/hft5/model-100.checkpoint
INFO:absl:eval/en_msa_translation/accuracy at step 100: 0.000
INFO:absl:eval/lav_msa_translation/accuracy at step 100: 0.000
INFO:absl:eval/mag_msa_translation/accuracy at step 100: 0.000
INFO:absl:Loading from /tmp/hft5/model-200.checkpoint
INFO:absl:eval/en_msa_translation/accuracy at step 200: 0.000
INFO:absl:eval/lav_msa_translation/accuracy at step 200: 0.000
INFO:absl:eval/mag_msa_translation/accuracy at step 200: 0.000
INFO:absl:Loading from /tmp/hft5/model-300.checkpoint
INFO:absl:eval/en_msa_translation/accuracy at step 300: 0.000
INFO:absl:eval/lav_msa_translation/accuracy at step 300: 0.000


##Predictions

In [0]:
inputs = [
    "translation: This is a totally valid sentence.",
    "translation: A doggy detail was walking famously.",
]
model.predict(
    inputs,
    sequence_length={"inputs": 32},
    batch_size=2,
    output_file="/tmp/hft5/example_predictions.txt",
)